## Switching Pattern Discovery
Section 5.2 in the paper

In [1]:
import os
wd = os.path.normpath(os.getcwd() + '/..')
os.chdir(wd)
os.getcwd()

'C:\\Users\\Fitogether\\Workspace\\research-footballcpd'

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sympy.combinatorics import Permutation
from sympy.interactive import init_printing
import matplotlib.pyplot as plt
from src.myconstants import *

In [3]:
# Choose an activity to analyze
activity_id = 245

In [11]:
fgp_path = f'data/fgp_avg/{activity_id}.csv'
fgp_df = pd.read_csv(fgp_path, header=0, encoding='utf-8-sig')
fgp_df['datetime'] = pd.to_datetime(fgp_df['datetime'])
fgp_df['roleperm'] = fgp_df.apply(lambda x: (x['base_role'], x['role']), axis=1)

form_path = f'data/form_avg/{activity_id}.pkl'
form_path = pd.read_pickle(form_path)

fgp_df

,player_id,squad_num,player_name,datetime,session,gametime,player_period,form_period,role_period,x,y,x_norm,y_norm,role,base_role,switch_rate,roleperm
0,4181,2,P02,2020-01-01 00:00:01,1,00:01.0,1,1,1,4667.0,512.0,-58.0,-2862.6,1,1,0.2,"(1, 1)"
1,4181,2,P02,2020-01-01 00:00:02,1,00:02.0,1,1,1,4694.0,388.0,-37.5,-2982.2,1,1,0.2,"(1, 1)"
2,4181,2,P02,2020-01-01 00:00:03,1,00:03.0,1,1,1,4657.0,315.0,-112.2,-3034.6,1,1,0.0,"(1, 1)"
3,4181,2,P02,2020-01-01 00:00:04,1,00:04.0,1,1,1,4681.0,340.0,-155.1,-3033.7,1,1,0.0,"(1, 1)"
4,4181,2,P02,2020-01-01 00:00:05,1,00:05.0,1,1,1,4718.0,440.0,-185.7,-3017.1,1,1,0.0,"(1, 1)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56352,4142,99,P99,2020-01-01 01:48:56,2,47:56.0,4,4,5,2624.0,1881.0,1072.1,-1456.9,10,10,0.5,"(10, 10)"
56353,4142,99,P99,2020-01-01 01:48:57,2,47:57.0,4,4,5,2438.0,1801.0,1010.5,-1469.9,10,10,0.5,"(10, 10)"
56354,4142,99,P99,2020-01-01 01:48:58,2,47:58.0,4,4,5,2269.0,1820.0,884.9,-1349.0,10,10,0.5,"(10, 10)"
56355,4142,99,P99,2020-01-01 01:48:59,2,47:59.0,4,4,5,2171.0,1959.0,781.4,-1169.7,10,10,0.5,"(10, 10)"


In [12]:
roleperms = fgp_df.pivot_table(
    values='roleperm', index='datetime', columns='player_name', aggfunc='first'
)
roleperms['switch_rate'] = fgp_df.groupby('datetime')['switch_rate'].first()
roleperms

player_name,P02,P06,P09,P10,P20,P23,P24,P30,P32,P42,P47,P70,P99,switch_rate
datetime,,,,,,,,,,,,,,
2020-01-01 00:00:01,"(1, 1)","(2, 2)","(3, 5)",NaN,"(4, 4)","(5, 3)","(6, 6)","(7, 7)",NaN,NaN,"(8, 8)","(9, 9)","(10, 10)",0.2
2020-01-01 00:00:02,"(1, 1)","(2, 2)","(3, 5)",NaN,"(4, 4)","(5, 3)","(6, 6)","(7, 7)",NaN,NaN,"(8, 8)","(9, 9)","(10, 10)",0.2
2020-01-01 00:00:03,"(1, 1)","(2, 2)","(3, 3)",NaN,"(4, 4)","(5, 5)","(6, 6)","(7, 7)",NaN,NaN,"(8, 8)","(9, 9)","(10, 10)",0.0
2020-01-01 00:00:04,"(1, 1)","(2, 2)","(3, 3)",NaN,"(4, 4)","(5, 5)","(6, 6)","(7, 7)",NaN,NaN,"(8, 8)","(9, 9)","(10, 10)",0.0
2020-01-01 00:00:05,"(1, 1)","(2, 2)","(3, 3)",NaN,"(4, 4)","(5, 5)","(6, 6)","(7, 7)",NaN,NaN,"(8, 8)","(9, 9)","(10, 10)",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01 01:48:56,"(1, 1)","(2, 4)",NaN,"(9, 3)",NaN,NaN,"(3, 6)","(7, 7)","(6, 9)","(4, 2)","(8, 8)","(5, 5)","(10, 10)",0.5
2020-01-01 01:48:57,"(1, 1)","(2, 4)",NaN,"(9, 3)",NaN,NaN,"(3, 6)","(7, 7)","(6, 9)","(4, 2)","(8, 8)","(5, 5)","(10, 10)",0.5
2020-01-01 01:48:58,"(1, 1)","(2, 4)",NaN,"(9, 3)",NaN,NaN,"(3, 6)","(7, 7)","(6, 9)","(4, 2)","(8, 8)","(5, 5)","(10, 10)",0.5


In [29]:
def decompose_to_cycles(roleperm):
    if roleperm['switch_rate'] > MAX_SWITCH_RATE:
        return []
    
    perm_list = [0] + [r for (l, r) in sorted([t for t in roleperm[:-1] if str(t) != 'nan'])]
    if len(perm_list) < 11:
        return []
    
    p = Permutation(perm_list)
    perm_str = str(p)
    
    result_list = []
    cycles_str = perm_str.split(')')
    for c in cycles_str:
        c = c.replace('(', '')
        result_list.append(c.split(' '))
        
    return [l for l in result_list if len(l) > 1]

In [28]:
cycles = roleperms.apply(decompose_to_cycles, axis=1)
cycle_list = [(dt, ' '.join(c)) for dt, cycles_t in cycles.iteritems() for c in cycles_t]
cycles_flatten = pd.Series([c for (dt, c) in cycle_list], index=[dt for (dt, c) in cycle_list])
cycles_flatten

2020-01-01 00:00:01         3 5
2020-01-01 00:00:02         3 5
2020-01-01 00:00:12    1 10 4 2
2020-01-01 00:00:13    1 10 4 2
2020-01-01 00:00:14    1 10 4 2
                         ...   
2020-01-01 01:48:58       3 6 9
2020-01-01 01:48:59         2 4
2020-01-01 01:48:59       3 6 9
2020-01-01 01:49:00         2 3
2020-01-01 01:49:00         6 9
Length: 4298, dtype: object